In [1]:
%load_ext autoreload
%autoreload 2

# Prepare Data

## Load Datasets

In [2]:
# Get data with our own functions, train & test splits
from WorkforceSentimentMonitoring.data import get_data, merge, holdout

submission, train, test = get_data()

display(submission.head())
display(train.head())
display(test.head())

,ID,overall
0,5,5
1,6,4
2,7,4
3,8,5
4,9,5


,ID,Place,location,date,status,job_title,summary,positives,negatives,advice_to_mgmt,score_1,score_2,score_3,score_4,score_5,score_6,overall
0,1,startup_1,NaN,"Dec 11, 2018",Current Employee,Anonymous Employee,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,NaN,4.0,5.0,5.0,4.0,5.0,0,5.0
1,2,startup_1,"Mountain View, CA","Jun 21, 2013",Former Employee,Program Manager,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,2.0,3.0,3.0,5.0,3.0,2094,5.0
2,3,startup_1,"New York, NY","May 10, 2014",Current Employee,Software Engineer III,Great balance between big-company security and...,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",Keep the focus on the user. Everything else wi...,5.0,4.0,5.0,5.0,4.0,949,5.0
3,4,startup_1,"Mountain View, CA","Feb 8, 2015",Current Employee,Anonymous Employee,The best place I've worked and also the most d...,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,Keep on NOT micromanaging - that is a huge ben...,2.0,5.0,5.0,4.0,5.0,498,4.0
4,10,startup_1,NaN,"Dec 9, 2018",Current Employee,Anonymous Employee,Execellent for engineers,Impact driven. Best tech in the world.,Size matters. Engineers are a bit disconnected...,NaN,5.0,5.0,5.0,5.0,5.0,0,4.0


,ID,Place,location,date,status,job_title,summary,positives,negatives,advice_to_mgmt,score_1,score_2,score_3,score_4,score_5,score_6
0,5,startup_1,"Los Angeles, CA","Jul 19, 2018",Former Employee,Software Engineer,"Unique, one of a kind dream job",Google is a world of its own. At every other c...,"If you don't work in MTV (HQ), you will be giv...",Promote managers into management for their man...,5,5,5,5,5,49
1,6,startup_1,"Mountain View, CA","Dec 9, 2018",Former Employee,SDE2,NICE working in GOOGLE as an INTERN,"People are not that busy, so they are nice to ...",Food is not good as I expected. People said it...,NaN,4,4,4,5,4,1
2,7,startup_1,"New York, NY","Dec 11, 2018",Current Employee,Software Engineer,Software engineer,Great working environment. Good work life balance,Usual big company problems. Hierarchy.,NaN,5,4,4,5,4,0
3,8,startup_1,NaN,"Dec 11, 2018",Former Employee,Anonymous Employee,great place to work and progress,"work culture, benefits, growth, people,",No cons that i can think of,NaN,5,5,5,5,5,0
4,9,startup_1,"New York, NY","Dec 10, 2018",Current Employee,Anonymous Employee,Google Surpasses Realistic Expectations,Great products. Vision you can feel good about...,Younger employees complaining about the compan...,NaN,5,5,5,5,5,0


In [3]:
# merge corresponding dataframes
df = merge(submission, train, test)

In [4]:
df.head()

,summary,positives,negatives,advice_to_mgmt,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall,review
0,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,NaN,4,5,5,4,5,5,Best Company to work for People are smart and ...
1,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,2,3,3,5,3,5,"Moving at the speed of light, burn out is inev..."
2,Great balance between big-company security and...,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",Keep the focus on the user. Everything else wi...,5,4,5,5,4,5,Great balance between big-company security and...
3,The best place I've worked and also the most d...,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,Keep on NOT micromanaging - that is a huge ben...,2,5,5,4,5,4,The best place I've worked and also the most d...
4,Execellent for engineers,Impact driven. Best tech in the world.,Size matters. Engineers are a bit disconnected...,NaN,5,5,5,5,5,4,Execellent for engineers Impact driven. Best t...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52815 entries, 0 to 29271
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   summary               52734 non-null  object
 1   positives             52815 non-null  object
 2   negatives             52814 non-null  object
 3   advice_to_mgmt        30132 non-null  object
 4   work-balance          52815 non-null  uint8 
 5   culture-values        52815 non-null  uint8 
 6   career-opportunities  52815 non-null  uint8 
 7   comp-benefits         52815 non-null  uint8 
 8   senior-mgmt           52815 non-null  uint8 
 9   overall               52815 non-null  uint8 
 10  review                52815 non-null  object
dtypes: object(5), uint8(6)
memory usage: 2.7+ MB


In [6]:
# small sample of dataframe ==> random 10_000 rows
small_df = df.sample(n=10_000, random_state=1, replace=True)

In [7]:
small_df.head()

,summary,positives,negatives,advice_to_mgmt,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall,review
9204,Terrible Company,Decent pay and health plan.,"Stressful work environment, very high quotas, ...",Stop treating your workers like trash.,1,1,1,2,1,1,Terrible Company Decent pay and health plan. S...
14938,Amazon is great to buy and sell products in th...,its really cool place to work with amazon,itsw great job to work .join with us.,make easy to post products in facebook .,5,5,5,5,5,5,Amazon is great to buy and sell products in th...
6640,Low paymasters,Good quality projects to work on.,Unrealistic deadlines and if you have a bad bo...,Dont bring in people at the wrong pay grade/le...,2,3,3,1,3,3,Low paymasters Good quality projects to work o...
8709,Buying,"Startup Feel, Fun Environment, Great Company","Long hours, Poor Management, Growth",NaN,2,3,3,3,2,2,"Buying Startup Feel, Fun Environment, Great Co..."
26452,Ok,"Smart people, good compensation, the opportuni...",A lack of good communication between the diffe...,NaN,3,3,4,5,3,3,"Ok Smart people, good compensation, the opport..."


In [8]:
# rest index to ordinal
small_df = small_df.reset_index(drop=True)

In [9]:
small_df.head()

,summary,positives,negatives,advice_to_mgmt,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall,review
0,Terrible Company,Decent pay and health plan.,"Stressful work environment, very high quotas, ...",Stop treating your workers like trash.,1,1,1,2,1,1,Terrible Company Decent pay and health plan. S...
1,Amazon is great to buy and sell products in th...,its really cool place to work with amazon,itsw great job to work .join with us.,make easy to post products in facebook .,5,5,5,5,5,5,Amazon is great to buy and sell products in th...
2,Low paymasters,Good quality projects to work on.,Unrealistic deadlines and if you have a bad bo...,Dont bring in people at the wrong pay grade/le...,2,3,3,1,3,3,Low paymasters Good quality projects to work o...
3,Buying,"Startup Feel, Fun Environment, Great Company","Long hours, Poor Management, Growth",NaN,2,3,3,3,2,2,"Buying Startup Feel, Fun Environment, Great Co..."
4,Ok,"Smart people, good compensation, the opportuni...",A lack of good communication between the diffe...,NaN,3,3,4,5,3,3,"Ok Smart people, good compensation, the opport..."


## Prepare Features & Targets

### Prepare Feature columns

In [10]:
# select out the text cols
text_columns = small_df.select_dtypes('object').columns
# dataframe of text columns
text_df = small_df[text_columns]
text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   summary         9992 non-null   object
 1   positives       10000 non-null  object
 2   negatives       10000 non-null  object
 3   advice_to_mgmt  5709 non-null   object
 4   review          10000 non-null  object
dtypes: object(5)
memory usage: 390.8+ KB


In [104]:
# import temporary feature engineering packages
from WorkforceSentimentMonitoring.trainer import Trainer, FEATURE_COLS, SCORE_COLS, feature_engineer, getSubjectivity, getPolarity, feature_sentiment, get_lengths, scaler, linear_feature
from WorkforceSentimentMonitoring.feature_engineering import getPolarity, getSubjectivity, get_subjectivity_polarity_columns, add_multinomial_nb_prediction_feature, get_linear_regression_cols, minmax_when_needed, train_logReg

In [105]:
from WorkforceSentimentMonitoring.preprocessing import preprocessing

In [12]:
text_df = text_df.astype('U').applymap(preprocessing)

In [13]:
# sample of preprocessed text columns
small_df[text_df.columns] = text_df
small_df.head()

,summary,positives,negatives,advice_to_mgmt,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall,review
0,terrible company,decent pay health plan,stressful work environment high quota worker s...,stop treating worker like trash,1,1,1,2,1,1,terrible company decent pay health plan stress...
1,amazon great buy sell product world,really cool place work amazon,itsw great job work join u,make easy post product facebook,5,5,5,5,5,5,amazon great buy sell product world really coo...
2,low paymaster,good quality project work,unrealistic deadline bad bos life tough,dont bring people wrong pay grade level lookin...,2,3,3,1,3,3,low paymaster good quality project work unreal...
3,buying,startup feel fun environment great company,long hour poor management growth,nan,2,3,3,3,2,2,buying startup feel fun environment great comp...
4,ok,smart people good compensation opportunity wor...,lack good communication different business org...,nan,3,3,4,5,3,3,ok smart people good compensation opportunity ...


#### Feature Engineering #1

In [15]:
# get lengths features
small_df = get_lengths(small_df)

In [16]:
small_df.head()

,summary,positives,negatives,advice_to_mgmt,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall,review,summary_length,postives_length,negatives_length,advice_length,combined_length
0,terrible company,decent pay health plan,stressful work environment high quota worker s...,stop treating worker like trash,1,1,1,2,1,1,terrible company decent pay health plan stress...,16,22,60,31,132
1,amazon great buy sell product world,really cool place work amazon,itsw great job work join u,make easy post product facebook,5,5,5,5,5,5,amazon great buy sell product world really coo...,35,29,26,31,124
2,low paymaster,good quality project work,unrealistic deadline bad bos life tough,dont bring people wrong pay grade level lookin...,2,3,3,1,3,3,low paymaster good quality project work unreal...,13,25,39,71,151
3,buying,startup feel fun environment great company,long hour poor management growth,nan,2,3,3,3,2,2,buying startup feel fun environment great comp...,6,42,32,3,86
4,ok,smart people good compensation opportunity wor...,lack good communication different business org...,nan,3,3,4,5,3,3,ok smart people good compensation opportunity ...,2,76,55,3,139


In [17]:
small_df = small_df[['summary', 'positives', 'negatives', 'advice_to_mgmt', 'review', 'work-balance','culture-values', 'career-opportunities', 'comp-benefits', 'senior-mgmt', 'overall', 'summary_length', 'postives_length', 'negatives_length', 'advice_length', 'combined_length']]
small_df.head()

,summary,positives,negatives,advice_to_mgmt,review,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall,summary_length,postives_length,negatives_length,advice_length,combined_length
0,terrible company,decent pay health plan,stressful work environment high quota worker s...,stop treating worker like trash,terrible company decent pay health plan stress...,1,1,1,2,1,1,16,22,60,31,132
1,amazon great buy sell product world,really cool place work amazon,itsw great job work join u,make easy post product facebook,amazon great buy sell product world really coo...,5,5,5,5,5,5,35,29,26,31,124
2,low paymaster,good quality project work,unrealistic deadline bad bos life tough,dont bring people wrong pay grade level lookin...,low paymaster good quality project work unreal...,2,3,3,1,3,3,13,25,39,71,151
3,buying,startup feel fun environment great company,long hour poor management growth,nan,buying startup feel fun environment great comp...,2,3,3,3,2,2,6,42,32,3,86
4,ok,smart people good compensation opportunity wor...,lack good communication different business org...,nan,ok smart people good compensation opportunity ...,3,3,4,5,3,3,2,76,55,3,139


In [18]:
# print(SCORE_COLS)

In [19]:
# rename the variable
df = small_df.copy()

In [20]:
df.head()

,summary,positives,negatives,advice_to_mgmt,review,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall,summary_length,postives_length,negatives_length,advice_length,combined_length
0,terrible company,decent pay health plan,stressful work environment high quota worker s...,stop treating worker like trash,terrible company decent pay health plan stress...,1,1,1,2,1,1,16,22,60,31,132
1,amazon great buy sell product world,really cool place work amazon,itsw great job work join u,make easy post product facebook,amazon great buy sell product world really coo...,5,5,5,5,5,5,35,29,26,31,124
2,low paymaster,good quality project work,unrealistic deadline bad bos life tough,dont bring people wrong pay grade level lookin...,low paymaster good quality project work unreal...,2,3,3,1,3,3,13,25,39,71,151
3,buying,startup feel fun environment great company,long hour poor management growth,nan,buying startup feel fun environment great comp...,2,3,3,3,2,2,6,42,32,3,86
4,ok,smart people good compensation opportunity wor...,lack good communication different business org...,nan,ok smart people good compensation opportunity ...,3,3,4,5,3,3,2,76,55,3,139


#### Train Text split

In [21]:
small_X_train, small_X_test, small_y_train, small_y_test = holdout(df, SCORE_COLS)

In [22]:
small_X_train

,summary,positives,negatives,advice_to_mgmt,review,summary_length,postives_length,negatives_length,advice_length,combined_length
0,good new hire,ambitious project work life balance energetic ...,lazy co worker bad canteen irregular hr,nan,good new hire ambitious project work life bala...,13,53,39,3,111
1,overall rating,account managment payment support mangment pay...,google adversary google work outsite field wor...,account managment,overall rating account managment payment suppo...,14,65,51,17,150
2,job,amazing team environment great work life balan...,experience outside apple weight lack consisten...,align consistent across board treat staff trea...,job amazing team environment great work life b...,3,143,144,165,458
3,great company overall pocket bad avoided,microsoft name technology someone microsoft wo...,microsoft bad reputation deserved perhaps way ...,stop naming thing modifier give shorter unique...,great company overall pocket bad avoided micro...,40,571,406,344,1364
4,know getting,nice coworkers easy get job get lot exercise d...,job stressful often hard body joint foot,increase everyone pay order get people stay jo...,know getting nice coworkers easy get job get l...,12,62,40,84,201
...,...,...,...,...,...,...,...,...,...,...
6995,great place work,management work environment benefit salary,really bad thing say,nan,great place work management work environment b...,16,42,20,3,84
6996,engaging ambiguous,always changing goal priority mean role varied...,unclear objective lead ambiguous review diffic...,remove bureaucracy promotion process relies he...,engaging ambiguous always changing goal priori...,18,55,93,82,251
6997,software engineer,achieve pretty good work life balance microsof...,company dying people disenchanted unimaginativ...,get rid people microsoft year least limit infl...,software engineer achieve pretty good work lif...,17,261,467,1363,2111
6998,great company lousy politics,great company great benefit working condition ...,much politics come making large design decisio...,nan,great company lousy politics great company gre...,28,82,97,3,213


In [23]:
small_y_train

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,5,5,3,5,5,4
1,4,4,4,4,5,4
2,4,3,2,2,2,3
3,4,5,5,4,3,3
4,3,1,4,2,3,4
...,...,...,...,...,...,...
6995,5,5,5,5,5,5
6996,3,4,4,4,3,3
6997,4,1,2,3,2,2
6998,3,4,3,5,3,3


#### Feature Engineering #2

In [24]:
# vectorized and NB to generate the features
small_X_train = add_multinomial_nb_prediction_feature(small_X_train, small_y_train)
small_X_test = add_multinomial_nb_prediction_feature(small_X_test, small_y_test)

In [25]:
small_X_train

,summary,positives,negatives,advice_to_mgmt,review,summary_length,postives_length,negatives_length,advice_length,combined_length,...,summary_senior-mgmt_nb,positives_senior-mgmt_nb,negatives_senior-mgmt_nb,advice_to_mgmt_senior-mgmt_nb,review_senior-mgmt_nb,summary_overall_nb,positives_overall_nb,negatives_overall_nb,advice_to_mgmt_overall_nb,review_overall_nb
0,good new hire,ambitious project work life balance energetic ...,lazy co worker bad canteen irregular hr,nan,good new hire ambitious project work life bala...,13,53,39,3,111,...,5,5,5,4,5,4,4,4,4,4
1,overall rating,account managment payment support mangment pay...,google adversary google work outsite field wor...,account managment,overall rating account managment payment suppo...,14,65,51,17,150,...,5,5,5,5,5,4,4,4,4,4
2,job,amazing team environment great work life balan...,experience outside apple weight lack consisten...,align consistent across board treat staff trea...,job amazing team environment great work life b...,3,143,144,165,458,...,4,4,3,3,4,4,4,4,4,4
3,great company overall pocket bad avoided,microsoft name technology someone microsoft wo...,microsoft bad reputation deserved perhaps way ...,stop naming thing modifier give shorter unique...,great company overall pocket bad avoided micro...,40,571,406,344,1364,...,3,3,3,3,3,3,3,3,3,3
4,know getting,nice coworkers easy get job get lot exercise d...,job stressful often hard body joint foot,increase everyone pay order get people stay jo...,know getting nice coworkers easy get job get l...,12,62,40,84,201,...,3,3,3,3,3,4,3,4,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,great place work,management work environment benefit salary,really bad thing say,nan,great place work management work environment b...,16,42,20,3,84,...,4,4,5,4,5,4,4,4,4,4
6996,engaging ambiguous,always changing goal priority mean role varied...,unclear objective lead ambiguous review diffic...,remove bureaucracy promotion process relies he...,engaging ambiguous always changing goal priori...,18,55,93,82,251,...,3,4,4,3,4,3,4,4,3,4
6997,software engineer,achieve pretty good work life balance microsof...,company dying people disenchanted unimaginativ...,get rid people microsoft year least limit infl...,software engineer achieve pretty good work lif...,17,261,467,1363,2111,...,4,3,3,2,3,4,4,3,3,3
6998,great company lousy politics,great company great benefit working condition ...,much politics come making large design decisio...,nan,great company lousy politics great company gre...,28,82,97,3,213,...,3,5,3,4,4,3,4,3,4,3


#### Feature Engineering #3

In [26]:
# extract subjectivity and polarity
small_X_train = get_subjectivity_polarity_columns(small_X_train)
small_X_test = get_subjectivity_polarity_columns(small_X_test)

100%|██████████| 5/5 [00:06<00:00,  1.20s/it]


In [27]:
small_X_train

,summary,positives,negatives,advice_to_mgmt,review,summary_length,postives_length,negatives_length,advice_length,combined_length,...,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,polarity_negatives,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review
0,good new hire,ambitious project work life balance energetic ...,lazy co worker bad canteen irregular hr,nan,good new hire ambitious project work life bala...,13,53,39,3,111,...,0.527273,0.418182,0.625000,0.375000,0.833333,-0.475000,0.000000,0.000000,0.661869,0.106061
1,overall rating,account managment payment support mangment pay...,google adversary google work outsite field wor...,account managment,overall rating account managment payment suppo...,14,65,51,17,150,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,job,amazing team environment great work life balan...,experience outside apple weight lack consisten...,align consistent across board treat staff trea...,job amazing team environment great work life b...,3,143,144,165,458,...,0.000000,0.000000,0.587500,0.350000,0.416667,-0.100000,0.616667,0.050000,0.545000,0.125000
3,great company overall pocket bad avoided,microsoft name technology someone microsoft wo...,microsoft bad reputation deserved perhaps way ...,stop naming thing modifier give shorter unique...,great company overall pocket bad avoided micro...,40,571,406,344,1364,...,0.472222,0.033333,0.391987,0.204808,0.563636,0.110227,0.627083,0.141004,0.506548,0.145801
4,know getting,nice coworkers easy get job get lot exercise d...,job stressful often hard body joint foot,increase everyone pay order get people stay jo...,know getting nice coworkers easy get job get l...,12,62,40,84,201,...,0.000000,0.000000,0.916667,0.516667,0.541667,-0.291667,0.000000,0.000000,0.791667,0.247222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,great place work,management work environment benefit salary,really bad thing say,nan,great place work management work environment b...,16,42,20,3,84,...,0.750000,0.800000,0.000000,0.000000,0.666667,-0.700000,0.000000,0.000000,0.708333,0.050000
6996,engaging ambiguous,always changing goal priority mean role varied...,unclear objective lead ambiguous review diffic...,remove bureaucracy promotion process relies he...,engaging ambiguous always changing goal priori...,18,55,93,82,251,...,0.700000,0.400000,0.693750,0.043750,0.611111,-0.022222,0.500000,-0.200000,0.631548,0.031548
6997,software engineer,achieve pretty good work life balance microsof...,company dying people disenchanted unimaginativ...,get rid people microsoft year least limit infl...,software engineer achieve pretty good work lif...,17,261,467,1363,2111,...,0.000000,0.000000,0.654028,0.244583,0.601471,0.049020,0.430247,0.062584,0.526733,0.096263
6998,great company lousy politics,great company great benefit working condition ...,much politics come making large design decisio...,nan,great company lousy politics great company gre...,28,82,97,3,213,...,0.625000,0.150000,0.800000,0.700000,0.314286,0.207143,0.000000,0.000000,0.611224,0.402041


#### Scaling & Feature Engineering #4

In [28]:
# minmaxscale the value to be centered around (0,1)
small_X_train, small_X_test = minmax_when_needed(small_X_train, small_X_test)

In [29]:
# drop non-numerical features
small_X_train.drop(columns=FEATURE_COLS, inplace=True)
small_X_test.drop(columns=FEATURE_COLS, inplace=True)

In [30]:
small_X_train

,summary_length,postives_length,negatives_length,advice_length,combined_length,summary_work-balance_nb,positives_work-balance_nb,negatives_work-balance_nb,advice_to_mgmt_work-balance_nb,review_work-balance_nb,...,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,polarity_negatives,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review
0,0.128713,0.022452,0.006865,0.001609,0.013251,1.00,1.00,1.00,0.75,1.00,...,0.527273,0.418182,0.625000,0.375000,0.833333,-0.475000,0.000000,0.000000,0.661869,0.106061
1,0.138614,0.027840,0.008977,0.009115,0.019260,0.75,0.75,0.75,0.75,0.75,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.029703,0.062865,0.025348,0.088472,0.066718,0.75,0.75,0.75,0.50,0.75,...,0.000000,0.000000,0.587500,0.350000,0.416667,-0.100000,0.616667,0.050000,0.545000,0.125000
3,0.396040,0.255052,0.071466,0.184450,0.206317,0.75,0.75,0.75,0.75,0.75,...,0.472222,0.033333,0.391987,0.204808,0.563636,0.110227,0.627083,0.141004,0.506548,0.145801
4,0.118812,0.026493,0.007041,0.045040,0.027119,0.50,0.50,0.50,0.50,0.50,...,0.000000,0.000000,0.916667,0.516667,0.541667,-0.291667,0.000000,0.000000,0.791667,0.247222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,0.158416,0.017512,0.003521,0.001609,0.009091,1.00,1.00,0.75,0.75,1.00,...,0.750000,0.800000,0.000000,0.000000,0.666667,-0.700000,0.000000,0.000000,0.708333,0.050000
6996,0.178218,0.023350,0.016370,0.043968,0.034823,0.50,0.75,0.75,0.50,0.50,...,0.700000,0.400000,0.693750,0.043750,0.611111,-0.022222,0.500000,-0.200000,0.631548,0.031548
6997,0.168317,0.115851,0.082204,0.730831,0.321418,0.75,0.75,0.75,0.75,0.75,...,0.000000,0.000000,0.654028,0.244583,0.601471,0.049020,0.430247,0.062584,0.526733,0.096263
6998,0.277228,0.035474,0.017074,0.001609,0.028968,0.50,0.50,0.75,0.75,0.50,...,0.625000,0.150000,0.800000,0.700000,0.314286,0.207143,0.000000,0.000000,0.611224,0.402041


In [31]:
small_X_train, small_X_test = get_linear_regression_cols(small_X_train, small_X_test, small_y_train)

100%|██████████| 6/6 [00:00<00:00, 24.49it/s]


In [32]:
small_X_train

,summary_length,postives_length,negatives_length,advice_length,combined_length,summary_work-balance_nb,positives_work-balance_nb,negatives_work-balance_nb,advice_to_mgmt_work-balance_nb,review_work-balance_nb,...,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review,work-balance_regression,culture-values_regression,career-opportunities_regression,comp-benefits_regression,senior-mgmt_regression,overall_regression
0,0.128713,0.022452,0.006865,0.001609,0.013251,1.00,1.00,1.00,0.75,1.00,...,0.000000,0.000000,0.661869,0.106061,4.908659,4.929658,4.275579,4.922897,4.588032,4.226959
1,0.138614,0.027840,0.008977,0.009115,0.019260,0.75,0.75,0.75,0.75,0.75,...,0.000000,0.000000,0.000000,0.000000,4.001545,4.296191,4.221158,3.884448,4.932942,4.547852
2,0.029703,0.062865,0.025348,0.088472,0.066718,0.75,0.75,0.75,0.50,0.75,...,0.616667,0.050000,0.545000,0.125000,3.122168,3.342215,2.797256,3.741536,2.625563,3.289185
3,0.396040,0.255052,0.071466,0.184450,0.206317,0.75,0.75,0.75,0.75,0.75,...,0.627083,0.141004,0.506548,0.145801,3.703233,3.683149,3.998842,3.469999,2.652609,2.662354
4,0.118812,0.026493,0.007041,0.045040,0.027119,0.50,0.50,0.50,0.50,0.50,...,0.000000,0.000000,0.791667,0.247222,1.987921,2.982082,2.514659,3.332644,2.289998,2.720184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,0.158416,0.017512,0.003521,0.001609,0.009091,1.00,1.00,0.75,0.75,1.00,...,0.000000,0.000000,0.708333,0.050000,4.296810,4.685642,4.564432,4.615076,4.275294,3.996674
6996,0.178218,0.023350,0.016370,0.043968,0.034823,0.50,0.75,0.75,0.50,0.50,...,0.500000,-0.200000,0.631548,0.031548,2.879164,3.404100,3.360514,3.622128,2.867048,2.999054
6997,0.168317,0.115851,0.082204,0.730831,0.321418,0.75,0.75,0.75,0.75,0.75,...,0.430247,0.062584,0.526733,0.096263,4.003470,3.118623,3.414131,4.643253,2.659120,3.139343
6998,0.277228,0.035474,0.017074,0.001609,0.028968,0.50,0.50,0.75,0.75,0.50,...,0.000000,0.000000,0.611224,0.402041,3.120543,3.707178,3.497243,4.368154,3.285279,3.145630


### Prepare Target columns

In [34]:
small_y_train

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,5,5,3,5,5,4
1,4,4,4,4,5,4
2,4,3,2,2,2,3
3,4,5,5,4,3,3
4,3,1,4,2,3,4
...,...,...,...,...,...,...
6995,5,5,5,5,5,5
6996,3,4,4,4,3,3
6997,4,1,2,3,2,2
6998,3,4,3,5,3,3


# Train Model

## LogReg with default params

In [40]:
from sklearn.linear_model import LogisticRegression
prediction_scores = {}
for target in SCORE_COLS:
    model = LogisticRegression(max_iter=10000)
    model.fit(small_X_train, small_y_train[target])
    prediction_scores[target] = model.score(small_X_test, small_y_test[target])

In [41]:
prediction_scores

{'work-balance': 0.661,
 'culture-values': 0.6136666666666667,
 'career-opportunities': 0.666,
 'comp-benefits': 0.6996666666666667,
 'senior-mgmt': 0.682,
 'overall': 0.6333333333333333}

In [43]:
from tqdm import tqdm
from time import sleep

In [47]:
train_log = train_logReg(small_X_train, small_y_train, small_X_test, small_y_test, score_cols=SCORE_COLS)

 17%|█▋        | 1/6 [00:03<00:15,  3.02s/it]/Users/yunxizzy/.pyenv/versions/3.7.7/envs/workforce/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
 83%|████████▎ | 5/6 [00:14<00:02,  2.92s/it]/Users/yunxizzy/.pyenv/versions/3.7.7/envs/workforce/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/module

In [45]:
for target in tqdm(SCORE_COLS):
    model = LogisticRegression(max_iter=10000)
    model.fit(small_X_train, small_y_train[target])
    prediction_scores[target] = model.score(small_X_test, small_y_test[target])

100%|██████████| 6/6 [00:19<00:00,  3.19s/it]


In [49]:
pred_dict = train_log[1]
pred_dict

{'work-balance': 0.661,
 'culture-values': 0.613,
 'career-opportunities': 0.666,
 'comp-benefits': 0.6996666666666667,
 'senior-mgmt': 0.682,
 'overall': 0.6333333333333333}

## RandomForest

### RandomForest with default params

In [50]:
train_rf = Trainer(small_X_train, small_X_test)

In [52]:
predictions, pred_scores = train_rf.train(small_y_train, small_y_test)

RandomForestClassifier
RandomForestClassifier
RandomForestClassifier
RandomForestClassifier
RandomForestClassifier
RandomForestClassifier


In [53]:
predictions

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,5,5,5,5,5,5
1,4,4,5,5,5,5
2,4,5,5,5,5,5
3,5,5,5,5,5,5
4,5,5,5,5,5,5
...,...,...,...,...,...,...
2995,1,1,1,5,1,3
2996,4,3,3,4,3,4
2997,3,4,5,4,3,4
2998,3,4,5,4,4,4


In [54]:
pred_scores

{'work-balance': 0.7716666666666666,
 'culture-values': 0.6883333333333334,
 'career-opportunities': 0.7546666666666667,
 'comp-benefits': 0.737,
 'senior-mgmt': 0.7913333333333333,
 'overall': 0.6976666666666667}

### RandomForest with selected params

In [106]:
trainer_rf_v1 = Trainer(small_X_train, small_X_test, estimator='RandomForest')

In [107]:
predictions, pred_scores = trainer_rf_v1.train(small_y_train, small_y_test)

RandomForestClassifier
              precision    recall  f1-score   support

           1       0.60      0.75      0.67       298
           2       0.39      0.51      0.44       316
           3       0.77      0.71      0.74       781
           4       0.81      0.72      0.76       898
           5       0.84      0.83      0.83       707

    accuracy                           0.72      3000
   macro avg       0.68      0.70      0.69      3000
weighted avg       0.74      0.72      0.73      3000

RandomForestClassifier
              precision    recall  f1-score   support

           1       0.60      0.54      0.57       314
           2       0.35      0.22      0.27       446
           3       0.44      0.37      0.40       600
           4       0.49      0.66      0.57       579
           5       0.73      0.80      0.77      1061

    accuracy                           0.57      3000
   macro avg       0.52      0.52      0.51      3000
weighted avg       0.56      0.

In [108]:
predictions

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,5,5,5,5,5,5
1,4,4,5,5,5,5
2,4,5,5,5,5,5
3,5,5,5,5,5,5
4,5,5,5,5,5,5
...,...,...,...,...,...,...
2995,1,1,1,1,1,1
2996,2,2,3,4,3,2
2997,3,3,5,4,3,4
2998,3,3,5,4,4,4


In [109]:
pred_scores

{'work-balance': 0.724,
 'culture-values': 0.575,
 'career-opportunities': 0.7093333333333334,
 'comp-benefits': 0.652,
 'senior-mgmt': 0.7333333333333333,
 'overall': 0.6343333333333333}

### Add GridSearch for RandomForest

In [ ]:
# params to tune: max_leaf

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier,

## AdaBoost

### AdaBoost with default params

In [79]:
trainer_ada_df = Trainer(small_X_train, small_X_test, estimator='AdaBoost')

In [80]:
predictions, pred_scores = trainer_ada_df.train(small_y_train, small_y_test)

AdaBoostClassifier
              precision    recall  f1-score   support

           1       0.57      0.64      0.60       335
           2       0.36      0.63      0.46       231
           3       0.80      0.73      0.77       791
           4       0.84      0.76      0.80       888
           5       0.87      0.80      0.83       755

    accuracy                           0.74      3000
   macro avg       0.69      0.71      0.69      3000
weighted avg       0.77      0.74      0.75      3000

AdaBoostClassifier
              precision    recall  f1-score   support

           1       0.51      0.47      0.49       306
           2       0.12      0.30      0.17       110
           3       0.47      0.59      0.52       399
           4       0.71      0.73      0.72       753
           5       0.89      0.73      0.80      1432

    accuracy                           0.67      3000
   macro avg       0.54      0.56      0.54      3000
weighted avg       0.72      0.67      

In [82]:
predictions

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,5,5,5,5,5,5
1,4,4,5,5,5,5
2,4,5,5,5,5,5
3,5,5,5,5,5,4
4,5,5,5,5,5,5
...,...,...,...,...,...,...
2995,1,1,2,5,1,1
2996,2,3,3,4,3,4
2997,3,4,5,4,3,4
2998,3,3,5,4,4,4


In [83]:
pred_scores

{'work-balance': 0.7403333333333333,
 'culture-values': 0.668,
 'career-opportunities': 0.7383333333333333,
 'comp-benefits': 0.731,
 'senior-mgmt': 0.726,
 'overall': 0.6966666666666667}

### AdaBoost with selected params

In [68]:
trainer_ada = Trainer(small_X_train, small_X_test, estimator='AdaBoost')

In [69]:
predictions, pred_scores = trainer_ada.train(small_y_train, small_y_test)

AdaBoostClassifier
AdaBoostClassifier
AdaBoostClassifier
AdaBoostClassifier
AdaBoostClassifier
AdaBoostClassifier


In [70]:
predictions

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,5,5,5,5,5,5
1,4,4,5,5,5,5
2,4,5,5,5,5,5
3,5,5,5,5,5,5
4,5,5,5,5,5,5
...,...,...,...,...,...,...
2995,1,1,1,1,1,1
2996,2,3,3,4,3,4
2997,3,4,5,4,3,4
2998,3,3,5,4,4,4


In [71]:
pred_scores

{'work-balance': 0.77,
 'culture-values': 0.671,
 'career-opportunities': 0.7646666666666667,
 'comp-benefits': 0.738,
 'senior-mgmt': 0.8046666666666666,
 'overall': 0.6886666666666666}

## XGB

### XGBClassifier with default params

In [72]:
trainer_xgb = Trainer(small_X_train, small_X_test, estimator='XGB')

In [73]:
predictions, pred_scores = trainer_xgb.train(small_y_train, small_y_test)

XGBClassifier
XGBClassifier
XGBClassifier
XGBClassifier
XGBClassifier
XGBClassifier


In [74]:
predictions

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,5,5,5,5,5,3
1,4,4,5,5,5,5
2,4,5,5,5,5,4
3,5,5,5,5,5,4
4,5,5,5,5,5,4
...,...,...,...,...,...,...
2995,1,1,1,5,1,3
2996,3,3,3,4,3,4
2997,3,4,5,4,3,4
2998,3,4,5,4,4,4


In [75]:
pred_scores

{'work-balance': 0.7866666666666666,
 'culture-values': 0.7036666666666667,
 'career-opportunities': 0.773,
 'comp-benefits': 0.7433333333333333,
 'senior-mgmt': 0.808,
 'overall': 0.7266666666666667}

## GradientBoost

### GradientBoost with default params

In [85]:
trainer_gb = Trainer(small_X_train, small_X_test, estimator='GradientBoost')

In [86]:
predictions, pred_scores = trainer_gb.train(small_y_train, small_y_test)

GradientBoostingClassifier
              precision    recall  f1-score   support

           1       0.62      0.90      0.74       258
           2       0.44      0.84      0.57       211
           3       0.89      0.72      0.79       890
           4       0.88      0.77      0.82       908
           5       0.87      0.82      0.84       733

    accuracy                           0.79      3000
   macro avg       0.74      0.81      0.75      3000
weighted avg       0.82      0.79      0.79      3000

GradientBoostingClassifier
              precision    recall  f1-score   support

           1       0.56      0.71      0.63       225
           2       0.28      0.46      0.35       170
           3       0.56      0.63      0.59       444
           4       0.72      0.73      0.72       771
           5       0.89      0.75      0.81      1390

    accuracy                           0.71      3000
   macro avg       0.60      0.65      0.62      3000
weighted avg       0.74

In [87]:
predictions

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,5,5,5,5,5,5
1,4,4,5,5,5,5
2,4,5,5,5,5,5
3,5,5,5,5,5,4
4,5,5,5,5,5,4
...,...,...,...,...,...,...
2995,1,1,1,5,1,3
2996,3,3,3,4,3,4
2997,3,4,5,4,3,4
2998,3,3,5,4,4,4


In [88]:
pred_scores

{'work-balance': 0.785,
 'culture-values': 0.7056666666666667,
 'career-opportunities': 0.775,
 'comp-benefits': 0.7476666666666667,
 'senior-mgmt': 0.8156666666666667,
 'overall': 0.7276666666666667}